In [1]:
### library import
from partial_view import Partial_views
import torch
import os 
import numpy as np 
from smplpytorch.pytorch.smpl_layer import SMPL_Layer
import time
from mapping import vertex_mapping, face_mapping
from pytorch3d.structures import Meshes
import open3d as o3d


In [2]:
### data import

camera = {
		"H" :  576,
		"W" :  640,
		"R" :  np.array([90,0,90]),
		"T" :  torch.tensor([[-0.0000, 0.2, 2.7000]]),
		"K" :  torch.Tensor([502.9671325683594,503.04168701171875,322.229736328125,329.3377685546875]),
}

# Loading AMASS data
dataset = 'ACCAD'
seq_lvl_1 = 'Female1General_c3d'
seq_lvl_2 = 'A7 - crouch_poses'
data_path = os.path.join('data', dataset, seq_lvl_1,seq_lvl_2)

d = np.load(data_path + '.npz')
relevant_poses = d['poses']

d_poses = torch.tensor(d['poses'][:,:72],dtype=torch.float32) 
	
n_sample = d_poses.shape[0]

In [3]:
folder = "test"
try:
	os.mkdir(folder)
except:
	pass

In [135]:
### In this demo, we only show for one shape
pose = d_poses[0].reshape(1,-1)
		
pv = Partial_views(pose_axisang=pose,betas=None,camera=camera)
pv.run(folder,"image"+str(1), show2D=False,save2D=False,show3D=False,save3D=False)

depth_points shape (12139, 3)


# Method 1 Vertex mapping

In [18]:
### vertex and face 
V1 = pv.vertices.numpy().reshape(-1,3)
print(V1.shape)
V2 = pv.pcd.points
V2 = np.asarray(V2)
print(V2.shape)
F  = pv.ref_faces.numpy().reshape(-1,3)
F = np.asarray(F,dtype=int)
print(F.shape)

(6890, 3)
(12139, 3)
(13776, 3)


In [136]:
idx1 = vertex_mapping(V1, V2)
print(idx1[:10])
pcd = o3d.geometry.PointCloud()
### add offset 0.2 to make difference from the original point cloud
pcd.points = o3d.utility.Vector3dVector(V1[idx1] + 0.2)

[1601 1602 1602 1601 1601 1602 1602 1602 1601 1602]


## Visualization comparing the original point cloud and the projection point cloud

In [24]:
o3d.visualization.draw_geometries([pcd,pv.pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Method 2 Face mapping

In [20]:
idx2, proj_p = face_mapping(V1,V2,F)

In [25]:
proj_p.shape

(12139, 3)

In [137]:
### V_F is the array containing the coordinate for each face
V_F = V1[F]

### V_F2 is the array containing the coordinate for the face where V2 matched
V_F2 = V_F[r]

In [139]:
l = proj_p.shape[0]
### add offset 0.2 to make difference from the original point cloud
### proj is the array for faces recovered from the barycentric coordinate
proj = np.asarray( [np.inner (proj_p[i] , V_F2[i].T) for i in range(l)]) + 0.2
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(proj)

## Visualization comparing the original point cloud and the projection point cloud

In [140]:
o3d.visualization.draw_geometries([pcd2,pv.pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
